# Set up

In [1]:
%reload_ext sql
%config SqlMagic.autocommit=False
%config SqlMagic.autolimit=0
%config SqlMagic.autopandas=True
%config SqlMagic.displaylimit=200

In [2]:
%sql trino://localhost:9090/cuebiq/

'Connected: @cuebiq/'

In [3]:
import time
import pandas as pd
from sqlalchemy import create_engine
from trino.dbapi import connect
from datetime import datetime, timedelta

# Schemas 
Here we created 3 schema:  
1. presence_data: mainly for test purpose 
2. pop_density: keep aggregated population density data from device location 
3. od_matrix: aggregated OD matrix from trajectory dataset, only cell has more than 10 unique user from dl have been selected here.

In [4]:
%%sql
show schemas from dedicated

 * trino://localhost:9090/cuebiq/
Done.


,Schema
0,censustocensus
1,dataflowtest
2,default
3,ds_data_for_complexity
4,ds_data_for_complexity_v2
5,ds_data_for_complexity_v3
6,information_schema
7,od_matrix
8,od_matrix20
9,od_matrix_10


In [8]:
# from sqlalchemy.engine import create_engine
# output_schema_name = 'presence_data'
# # output_schema_name = 'pop_density'
# # output_schema_name = 'od_matrix'
# # output_schema_name = "20OD_matrix"

# con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

In [5]:
%%sql
show tables from dedicated.pop_density20

 * trino://localhost:9090/cuebiq/
Done.


,Table
0,pd_co_20200101_agg3
1,pd_co_20200101_agg5
2,pd_co_20200102_agg3
3,pd_co_20200102_agg5
4,pd_co_20200103_agg3
...,...
1013,pd_mx_20201008_agg3
1014,pd_mx_20201009_agg3
1015,pd_mx_20201010_agg3
1016,pd_mx_20201011_agg3


In [4]:
%%sql
show tables from dedicated.od_matrix_10

 * trino://localhost:9090/cuebiq/
Done.


,Table
0,od_co_20191101_agg3_10
1,od_co_20191102_agg3_10
2,od_co_20191103_agg3_10
3,od_co_20191104_agg3_10
4,od_co_20191105_agg3_10
...,...
2248,od_mx_20191230_agg3_10
2249,od_mx_20191230_agg5_10
2250,od_mx_20191231_agg3_10
2251,od_mx_20191231_agg5_10


In [8]:
%%sql
show tables from dedicated.presence_data

 * trino://localhost:9090/cuebiq/
Done.


,Table
0,od_id_20190101_agg3
1,od_id_20190101_agg5
2,od_id_20190102_agg3
3,od_id_20190102_agg5
4,od_id_20190103_agg3
...,...
445,pd_id_2019_agg3
446,pd_id_2019_agg5
447,pd_in_20190101_agg3
448,pd_in_20190102_agg3


In [68]:
%%sql
show tables from dedicated.pop_density20 LIKE 'pd_co_2020%_agg3'

 * trino://localhost:9090/cuebiq/
Done.


,Table
0,pd_co_20200101_agg3
1,pd_co_20200102_agg3
2,pd_co_20200103_agg3
3,pd_co_20200104_agg3
4,pd_co_20200105_agg3
...,...
361,pd_co_20201227_agg3
362,pd_co_20201228_agg3
363,pd_co_20201229_agg3
364,pd_co_20201230_agg3


## Create schema

In [11]:
output_schema_name = 'OD_matrix_10'

In [12]:
%sql create schema if not exists dedicated.$output_schema_name

 * trino://localhost:9090/cuebiq/
Done.


""


## Create table 

In [ ]:
# output_schema_name = 'presence_data'
# table_name = f"dedicated.{output_schema_name}.new_table_demo"

In [ ]:
# %sql create table if not exists $table_name (col_0 varchar, col_1 bigint, col_2 varchar)

In [10]:
# %sql show tables from dedicated.presence_data

## Delete table

In [39]:
# %%sql
# DROP TABLE IF EXISTS dedicated.presence_data.od_in_20190304_agg5_10

# # pop_density presence_data od_matrix

## Rename table

In [89]:
# Rename the table to the final name based on date and country code

output_schema_name = 'od_matrix'
con = create_engine(f"trino://localhost:9090/dedicated/{output_schema_name}")

rename_table_query = f"""
ALTER TABLE "od_id_20190301_test_agg5" RENAME TO "od_id_20190301_agg5"
"""

# Execute the rename table query
with con.connect() as connection:
    connection.execute(rename_table_query)

OperationalError: (trino.exceptions.TrinoExternalError) TrinoExternalError(type=EXTERNAL, name=HIVE_METASTORE_ERROR, message="New location for this table hive.od_matrix.od_id_20190301_agg5 already exists : s3a://cuebiq-paas-org-1872-kr7pteibwb/metastore/od_matrix.db/od_id_20190301_agg5", query_id=20240606_145713_00482_44guw)
[SQL: 
ALTER TABLE "od_id_20190301_test_agg5" RENAME TO "od_id_20190301_agg5"
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

## Check table

In [41]:
# check
# q2 = f"select * from dedicated.presence_data.od_id_20190103_agg3"
q2 = f"select * from dedicated.presence_data.od_in_20190305_agg5_10"

%sql $q2

 * trino://localhost:9090/cuebiq/
Done.


,start_geohash5,end_geohash5,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no
0,tfcm0,tfcm0,10,76.395000,49.133333,75.592505,84155.840447,1448.083967,176123.434667,4.700000,4.0,2.359378
1,tfcv1,tfcv1,11,48.259091,29.200000,41.487591,1293.730880,337.087539,2013.628653,3.909091,4.0,2.119177
2,teh0s,teh0s,31,76.482258,40.983333,87.158813,3176.713035,1085.400892,7060.047421,4.741935,4.0,2.999642
3,tehx5,tehx5,18,93.570370,50.008333,147.837689,5932.736450,4150.251032,4620.136758,7.166667,6.0,5.272013
4,ttn9m,ttn9m,29,56.191954,33.333333,101.350055,939.470212,386.404802,1981.120118,4.103448,4.0,2.041443
...,...,...,...,...,...,...,...,...,...,...,...,...
4333,ttnf9,ttnff,20,74.515000,48.350000,67.059044,8623.091227,7235.458201,5022.171754,8.700000,7.5,5.120650
4334,ts1s3,ts1s3,34,85.753922,42.891667,89.821224,3786.099083,1255.230018,7864.933845,5.794118,5.0,4.360841
4335,ts5dc,ts5df,33,64.837879,28.900000,114.732273,6536.246939,4547.360544,9962.828555,7.303030,5.0,8.741013
4336,ts5df,ts5df,314,82.483705,30.116667,141.250773,2379.624612,855.502570,4641.474093,5.722930,5.0,3.951339


In [91]:
# check
q2 = f"select * from dedicated.od_matrix_10.od_in_20191230_agg3_10"
%sql $q2

 * trino://localhost:9090/cuebiq/
Done.


,start_geohash3,end_geohash3,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no
0,t9q,t9q,168,90.433631,37.825000,150.608086,12977.103441,1232.043358,84596.342460,5.273810,4.0,4.424621
1,t9q,t9w,25,195.936000,155.566667,164.516324,34954.855695,22969.525325,36961.759188,11.680000,9.0,8.634813
2,t9r,t9r,147,93.138095,40.816667,133.600158,34810.359840,1246.073636,329514.843211,5.836735,4.0,9.881518
3,t9v,t9v,180,75.643333,33.641667,109.195463,7834.652322,1746.843723,22313.527141,6.383333,4.5,8.244365
4,t9v,t9y,23,135.369565,89.233333,124.927958,37901.986826,11037.737132,40742.713920,9.260870,6.0,10.445605
...,...,...,...,...,...,...,...,...,...,...,...,...
310,whd,whd,272,108.131556,45.900000,152.346627,6775.366603,1307.026094,15849.556446,5.893382,4.0,4.993689
311,whe,whe,43,154.191860,124.100000,160.017353,3111.001068,706.355942,7915.305467,4.279070,3.0,3.088578
312,whf,whf,76,101.727851,40.841667,133.132686,2582.323484,901.462119,4327.406123,5.960526,5.0,4.358718
313,whg,whg,290,97.844655,51.216667,129.384715,8101.150093,1532.315792,18460.816161,4.827586,4.0,3.579729


In [5]:
# check
q1 = f"select * from dedicated.pop_density.pd_in_20190401_agg3_2"
%sql $q1

 * trino://localhost:9090/cuebiq/
Done.


,geohash3,no_of_points,no_of_unique_users,local_date
0,tdk,788,42,20190401
1,tds,57071,1348,20190401
2,tdu,127440,2809,20190401
3,teh,62864,1600,20190401
4,tek,604563,13473,20190401
5,tes,110083,2409,20190401
6,teu,145817,3421,20190401
7,tsh,52185,1211,20190401
8,tsk,48399,1124,20190401
9,tss,29200,727,20190401


Check duplicates

In [8]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import time

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [9]:
# Construct the SQL query
query = f"""
SELECT DISTINCT *
FROM dedicated.pop_density20.pd_co_2020_agg3
"""

# query = """
# SELECT DISTINCT
#     start_geohash5,
#     end_geohash5,
#     trip_count,
#     ROUND(m_duration_min, 6) AS rounded_m_duration_min,
#     ROUND(mdn_duration_min, 6) AS rounded_mdn_duration_min,
#     ROUND(sd_duration_min, 6) AS rounded_sd_duration_min,
#     ROUND(m_length_m, 6) AS rounded_m_length_m,
#     ROUND(mdn_length_m, 6) AS rounded_mdn_length_m,
#     ROUND(sd_length_m, 6) AS rounded_sd_length_m,
#     ROUND(m_points_no, 6) AS rounded_m_points_no,
#     ROUND(mdn_points_no, 6) AS rounded_mdn_points_no,
#     ROUND(sd_points_no, 6) AS rounded_sd_points_no
# FROM
#     dedicated.presence_data.od_in_20190314_agg5_10
# """

# od_matrix pop_density
pe_dl_table_gen = sql_engine.read_sql(query)
pe_dl_table_gen

,geohash_3,no_of_points,no_of_unique_users,local_date
0,d35,16467,551,20200109
1,d20,3176,110,20200408
2,d21,8238,300,20200408
3,d31,2512,89,20200408
4,d34,94886,3397,20200408
...,...,...,...,...
14623,d2u,2232,100,20201224
14624,d33,2157,73,20201224
14625,d3e,1835,74,20201224
14626,6rg,3217,1779,20201222


In [10]:
# pe_dl_table_gen1 = pe_dl_table_gen.drop(columns=['partition_key'])
pe_dl_table_gen1 = pe_dl_table_gen
pe_dl_table_gen1

,geohash_3,no_of_points,no_of_unique_users,local_date
0,d35,16467,551,20200109
1,d20,3176,110,20200408
2,d21,8238,300,20200408
3,d31,2512,89,20200408
4,d34,94886,3397,20200408
...,...,...,...,...
14623,d2u,2232,100,20201224
14624,d33,2157,73,20201224
14625,d3e,1835,74,20201224
14626,6rg,3217,1779,20201222


In [11]:
pe_dl_table_gen1.columns

Index(['geohash_3', 'no_of_points', 'no_of_unique_users', 'local_date'], dtype='object')

In [12]:
df_no_duplicates = pe_dl_table_gen1.drop_duplicates()
# df_no_duplicates.sort_values('no_of_unique_users')
len(df_no_duplicates)

14628

In [13]:
df_no_duplicates

,geohash_3,no_of_points,no_of_unique_users,local_date
0,d35,16467,551,20200109
1,d20,3176,110,20200408
2,d21,8238,300,20200408
3,d31,2512,89,20200408
4,d34,94886,3397,20200408
...,...,...,...,...
14623,d2u,2232,100,20201224
14624,d33,2157,73,20201224
14625,d3e,1835,74,20201224
14626,6rg,3217,1779,20201222


In [73]:
# Filter rows where start_geohash5 and end_geohash5 are both 'tsvcq'
filtered_rows = pe_dl_table_gen[(pe_dl_table_gen['start_geohash5'] == 'tsvcq') & (pe_dl_table_gen['end_geohash5'] == 'tsvcq')]
filtered_rows

KeyError: 'start_geohash5'

In [34]:
# Round floating-point columns to a consistent number of decimal places
float_cols = ['m_duration_min', 'mdn_duration_min', 'sd_duration_min', 'm_length_m', 'mdn_length_m', 'sd_length_m', 'm_points_no', 'mdn_points_no', 'sd_points_no']
pe_dl_table_gen[float_cols] = pe_dl_table_gen[float_cols].round(6)
pe_dl_table_gen

,start_geohash5,end_geohash5,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no
0,tsvcq,tsvcq,11,128.877273,62.900000,127.898043,2354.096383,372.672212,4085.589320,3.000000,3.0,1.264911
1,tsz6r,tsz6r,30,119.378889,72.416667,110.041957,3421.426687,1361.605677,5382.658629,5.966667,5.0,3.458955
2,tepep,tepdz,28,57.676190,35.083333,65.592265,4796.820484,3965.895677,4354.375231,6.107143,5.5,3.489023
3,tg0w7,tg0w7,52,98.088141,56.883333,113.981592,2110.067269,914.110714,3001.185633,5.269231,4.5,3.581627
4,tsph4,tsph6,23,102.242754,67.666667,92.283766,6979.532405,4797.070849,6621.091930,6.086957,4.0,6.067147
...,...,...,...,...,...,...,...,...,...,...,...,...
10256,tepf9,tepf3,18,39.807407,36.041667,24.103395,3256.874778,2973.420258,2032.176075,5.388889,4.5,2.615314
10257,t9yb2,t9yb2,16,92.103125,56.025000,125.036038,1085.588826,412.583382,2246.769514,4.937500,5.0,3.065262
10258,tf346,tf343,68,49.219853,32.741667,51.828814,4556.405593,3978.065887,3966.518486,5.588235,4.0,3.546010
10259,t9yxn,t9yxn,21,136.998413,75.050000,206.412152,1483.551261,731.409189,1909.787484,5.380952,3.0,5.054465


In [38]:
df_deduplicated = pe_dl_table_gen.drop_duplicates()
df_deduplicated

,start_geohash3,end_geohash3,trip_count,m_duration_min,mdn_duration_min,sd_duration_min,m_length_m,mdn_length_m,sd_length_m,m_points_no,mdn_points_no,sd_points_no,local_date
0,d20,d20,169,62.178008,29.816667,88.641090,2662.541931,876.739984,7567.013864,5.230769,4.0,3.311236,20191221
1,d2f,d2g,28,285.432143,243.925000,245.766285,161772.245716,113306.635345,219836.659813,61.000000,24.0,182.479426,20191221
2,d2g,d34,12,375.676389,240.600000,310.954336,227053.121007,232135.140060,43071.708732,19.166667,13.0,21.302298,20191228
3,d35,d37,18,172.637963,146.275000,160.699512,85113.870760,74282.716433,68564.805729,23.833333,18.0,23.311825,20191228
4,d3d,d3f,14,373.089286,273.666667,244.587232,198825.896363,162365.523458,138877.736713,30.642857,25.5,26.243262,20191228
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4611,d3h,d3k,44,144.735606,112.108333,132.204439,21055.291804,16630.545199,12662.249058,8.659091,7.5,6.368320,20191124
4612,d2d,d2f,21,159.030159,81.550000,248.705326,50976.709312,50154.466785,61458.420424,6.095238,5.0,3.389760,20191219
4613,d33,d33,228,70.120249,31.633333,107.252807,5700.083571,993.826635,12612.235684,5.078947,4.0,3.862493,20191219
4614,d2f,d2c,14,169.389286,127.650000,167.332578,47057.113910,35594.389738,53085.624231,6.000000,5.0,3.234430,20191227


In [18]:
pe_dl_table_gen['local_date'].nunique()

362

In [15]:
unique_values_count = pe_dl_table_gen["local_date"].unique()
# unique_values_count = pe_dl_table_gen["local_date"].nunique()
unique_values_count

array(['20200109', '20200408', '20200223', '20200219', '20200206',
       '20200322', '20200115', '20200108', '20200216', '20200201',
       '20200224', '20200319', '20200304', '20200401', '20200125',
       '20200326', '20200221', '20200114', '20200113', '20200324',
       '20200327', '20200317', '20200313', '20200218', '20200119',
       '20200312', '20200103', '20200309', '20200129', '20200107',
       '20200211', '20200331', '20200301', '20200209', '20200305',
       '20200120', '20200212', '20200402', '20200323', '20200210',
       '20200320', '20200405', '20200116', '20200215', '20200131',
       '20200329', '20200428', '20200828', '20200702', '20200226',
       '20200105', '20200227', '20201005', '20200308', '20200111',
       '20200621', '20200307', '20201019', '20200203', '20200110',
       '20200403', '20200222', '20200409', '20200208', '20200122',
       '20200411', '20200321', '20200311', '20200123', '20200325',
       '20200316', '20200306', '20200117', '20200128', '202002

In [17]:
unique_values_count.sort()
unique_values_count

array(['20200101', '20200102', '20200103', '20200105', '20200106',
       '20200107', '20200108', '20200109', '20200110', '20200111',
       '20200112', '20200113', '20200114', '20200115', '20200116',
       '20200117', '20200118', '20200119', '20200120', '20200121',
       '20200122', '20200123', '20200124', '20200125', '20200126',
       '20200127', '20200128', '20200129', '20200131', '20200201',
       '20200202', '20200203', '20200204', '20200205', '20200206',
       '20200207', '20200208', '20200209', '20200210', '20200211',
       '20200212', '20200213', '20200214', '20200215', '20200216',
       '20200217', '20200218', '20200219', '20200220', '20200221',
       '20200222', '20200223', '20200224', '20200225', '20200226',
       '20200227', '20200228', '20200229', '20200301', '20200302',
       '20200303', '20200304', '20200305', '20200306', '20200307',
       '20200308', '20200309', '20200310', '20200311', '20200312',
       '20200313', '20200314', '20200315', '20200316', '202003

In [20]:
# Generate all dates in 2020
all_dates_2020 = pd.date_range(start='2020-01-01', end='2020-12-31')

# Find the missing dates
missing_dates = all_dates_2020.difference(unique_values_count)

# Display the missing dates
missing_dates = missing_dates.strftime('%Y%m%d')
missing_dates

Index(['20200104', '20200130', '20200429', '20200612'], dtype='object')

In [21]:
# # Has error

# sorted_dates = sorted(unique_values_count)

# # Function to find the missing date
# def find_missing_date(sorted_dates):
#     date_format = "%Y%m%d"
    
#     for i in range(len(sorted_dates) - 1):
#         current_date = datetime.strptime(sorted_dates[i], date_format)
#         next_date = datetime.strptime(sorted_dates[i + 1], date_format)
        
#         if next_date != current_date + timedelta(days=1):
#             return (current_date + timedelta(days=1)).strftime(date_format)
    
#     return None

# # Finding the missing date
# missing_date = find_missing_date(sorted_dates)
# print(f"The missing date is: {missing_date}")

The missing date is: 20200104
